In [ ]:
import dataclasses
import json
import pathlib
import subprocess
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [ ]:
process_result = subprocess.run(
    [
        "bazel",
        "run",
        "@rules_dotnet//tools/paket2bazel",
        "--",
        "--dependencies-file",
        str(pathlib.Path.cwd() / "paket.dependencies"),
        "--output-folder",
        str(pathlib.Path.cwd() / "deps"),
    ],
    capture_output=True,
    text=True,
)

In [ ]:
process_result = subprocess.run(
    ["bazel", "build", "//..."], check=True, capture_output=True
)
print(process_result.stdout.decode())

In [ ]:
process_result = subprocess.run(
    ["bazel", "aquery", "//...", "--output=jsonproto"], check=True, capture_output=True
)
aquery = json.loads(process_result.stdout)

In [ ]:
pathlib.Path("aquery.json").write_text(json.dumps(aquery, indent=2))

In [ ]:
@dataclasses.dataclass
class Action:
    target_id: int
    inputs: list[pathlib.Path]
    outputs: list[pathlib.Path]
    primary_output: pathlib.Path


def get_path(path_fragments: list[dict], path_fragment_id: int):
    fragment = (fragment,) = [
        other_fragment
        for other_fragment in path_fragments
        if other_fragment["id"] == path_fragment_id
    ]
    related_fragments: list[str] = [fragment["label"]]
    while "parentId" in fragment:
        (fragment,) = [
            other_fragment
            for other_fragment in path_fragments
            if other_fragment["id"] == fragment["parentId"]
        ]
        related_fragments.append(fragment["label"])

    path = pathlib.Path(related_fragments.pop())
    while related_fragments:
        path = path / related_fragments.pop()
    return path


def get_artifact_path(artifact_id: int):
    (artifact,) = [
        artifact for artifact in aquery["artifacts"] if artifact["id"] == artifact_id
    ]
    path_fragment_id = artifact["pathFragmentId"]
    path = get_path(aquery["pathFragments"], path_fragment_id)
    return path


def recursively_get_inputs(dep_set_id: int) -> list[pathlib.Path]:
    inputs = []
    (dep_set_of_files,) = [
        dep_set_of_files
        for dep_set_of_files in aquery["depSetOfFiles"]
        if dep_set_of_files["id"] == dep_set_id
    ]
    direct_artifact_ids = dep_set_of_files["directArtifactIds"]
    for direct_artifact_id in direct_artifact_ids:
        path = get_artifact_path(direct_artifact_id)
        inputs.append(path)
    for transitive_dep_set_id in dep_set_of_files.get("transitiveDepSetIds", []):
        inputs.extend(recursively_get_inputs(transitive_dep_set_id))
    return inputs


def get_action(target_id: int) -> Action:
    inputs: list[pathlib.Path] = []
    outputs: list[pathlib.Path] = []
    (action,) = [
        action
        for action in aquery["actions"]
        # A targetId can correspond to multiple actions:
        if action["targetId"] == target_id and action["mnemonic"] == "CSharpCompile"
    ]
    input_dep_set_ids = action["inputDepSetIds"]
    for input_dep_set_id in input_dep_set_ids:
        (dep_set_of_files,) = [
            dep_set_of_files
            for dep_set_of_files in aquery["depSetOfFiles"]
            if dep_set_of_files["id"] == input_dep_set_id
        ]

        direct_artifact_ids = dep_set_of_files["directArtifactIds"]
        for direct_artifact_id in direct_artifact_ids:
            path = get_artifact_path(direct_artifact_id)
            inputs.append(path)

        transitive_dep_set_ids = dep_set_of_files.get("transitiveDepSetIds", [])
        for transitive_dep_set_id in transitive_dep_set_ids:
            paths = recursively_get_inputs(transitive_dep_set_id)
            inputs.extend(paths)

    output_ids = action["outputIds"]
    for output_id in output_ids:
        path = get_artifact_path(output_id)
        outputs.append(path)

    primary_output_id = action["primaryOutputId"]
    (primary_output_artifact,) = [
        artifact
        for artifact in aquery["artifacts"]
        if artifact["id"] == primary_output_id
    ]
    primary_output_path_fragment_id = primary_output_artifact["pathFragmentId"]
    primary_output_path = get_path(
        aquery["pathFragments"], primary_output_path_fragment_id
    )

    (dotnet_cli_home,) = [
        env_var["value"]
        for env_var in action["environmentVariables"]
        if env_var["key"] == "DOTNET_CLI_HOME"
    ]
    dotnet_cli_home_path = pathlib.Path(dotnet_cli_home)
    inputs = [
        input_path
        for input_path in inputs
        if not input_path.is_relative_to(dotnet_cli_home_path)
    ]
    return Action(
        target_id=target_id,
        inputs=inputs,
        outputs=outputs,
        primary_output=primary_output_path,
    )


actions = []
for aquery_action in aquery["actions"]:
    if aquery_action["mnemonic"] == "CSharpCompile":
        action = get_action(aquery_action["targetId"])
        actions.append(action)

In [ ]:
for artifacts in aquery["artifacts"]:
    print(artifacts["id"], get_artifact_path(artifacts["id"]))

In [ ]:
output_base = subprocess.run(
    ["bazel", "info", "output_base"], capture_output=True, text=True
).stdout.strip()
print(f"output_base = {output_base}")
external_path = pathlib.Path(output_base) / "external"

In [ ]:
if not external_path.exists():
    raise FileNotFoundError("No external path found")
external_path = external_path.resolve()

In [ ]:
def find_external_path(path: pathlib.Path) -> pathlib.Path | None:
    full_path = external_path.parent / path
    if full_path.exists():
        return full_path
    raise FileNotFoundError(f"External path not found: {full_path}")


for action in actions:
    print("Action ID:", action.target_id)
    for inp in action.inputs:
        if str(inp.parents[-2]) == "external":
            print("Input", find_external_path(inp))
    for out in action.outputs:
        print("Output", out)
    print("Primary Output", action.primary_output)

In [ ]:
dlls = []
for action in actions:
    for path in action.inputs + action.outputs:
        if str(path.parents[-2]) == "external" and (
            (full_path := find_external_path(path)) is not None
        ):
            if full_path.is_file() and full_path.suffix == ".dll":
                dlls.append(full_path)
            if full_path.is_dir():
                raise ValueError("Directory found, unexpected")

dlls = list(set(dlls))

In [ ]:
def generate_csproj(
    csproj_path, source_files, dll_references, target_framework="net6.0"
):
    project = ET.Element("Project", Sdk="Microsoft.NET.Sdk")
    prop_group = ET.SubElement(project, "PropertyGroup")
    tf = ET.SubElement(prop_group, "TargetFramework")
    tf.text = target_framework

    item_group_sources = ET.SubElement(project, "ItemGroup")
    for src in source_files:
        ET.SubElement(item_group_sources, "Compile", Include=src)

    item_group_refs = ET.SubElement(project, "ItemGroup")
    for dll in dll_references:
        ref = ET.SubElement(
            item_group_refs, "Reference", Include=dll.split("/")[-1].replace(".dll", "")
        )
        hint = ET.SubElement(ref, "HintPath")
        hint.text = dll

    xmlstr = minidom.parseString(ET.tostring(project)).toprettyxml(indent="  ")
    with open(csproj_path, "w") as f:
        f.write(xmlstr)


generate_csproj("Project.csproj", [], list(map(str, dlls)))